In [ ]:
import utils
import loader
import graphics
import pandas as pd
import numpy as np

# Usar las trazas de Simva o las de Scorm
use_scorm = False

files_path = "./trazas/simva/"
cols_to_drop = ["stored", "id", "version", "actor.account.homePage", "authority.name", "authority.homePage", "context.contextActivities.category", "context.registration", "object.definition.name.en-US", "object.definition.description.en-US", "object.definition.type", "verb.display.en-US", "verb.id", "object.objectType", "result.response", "result.score.scaled", "result.completion", "result.success"]

if use_scorm:
	files_path = "./trazas/scorm/"
	cols_to_drop = ["verb.display.en-US", "id", "stored", "version", "actor.objectType", "actor.account.homePage", "result.success", "result.completion", "context.registration", "authority.objectType", "authority.account.homePage", "authority.account.name", "authority.name", "object.definition.description.en-US", "object.definition.name.en-US", "object.objectType", "context.contextActivities.category"]

files_extension = "json"


# Usuarios que se usaron para probar el estudio
users_to_drop = ["682b4a72c76d2e0023ed4d05_igbm", "682b4a41c76d2e0023ed4b24_dmyj", "682b4a41c76d2e0023ed4b24_sxif", "682b4a72c76d2e0023ed4d05_agog"]

In [ ]:
############################
# Datos comunes
# Sacado de los JSONs
############################
all_users_df, users_individual_df_list = loader.load_all_files(files_path, files_extension, "timestamp", cols_to_drop, use_scorm)
n_users = len(users_individual_df_list)


# Eliminar usuarios invalidos
all_users_df = all_users_df[~all_users_df['actor.account.name'].isin(users_to_drop)]
all_users_df = all_users_df.reset_index(drop=True)

users_to_drop = set(users_to_drop)
filtered_df_list = [
    df for df in users_individual_df_list 
    if df["actor.account.name"].unique()[0] not in users_to_drop
]
users_individual_df_list = filtered_df_list


game_starts_conditions = [("object.id", "GameStart")]
game_starts = utils.find_indices_by_conditions(all_users_df, game_starts_conditions)
game_starts = all_users_df.iloc[game_starts]
game_starts = game_starts.drop_duplicates(subset=["actor.account.name"])

gender_sexuality_combinations = [
	('male', 'heterosexual'), 
	('male', 'homosexual'), 
	('male', 'bisexual'), 
	('female', 'heterosexual'), 
	('female', 'homosexual'), 
	('female', 'bisexual')
]
demography = game_starts[["actor.account.name", "Gender", "Sexuality"]].values.tolist()

demography_info = {}
for user in demography:
	demography_info.update({user[0]: (user[1], user[2])})

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

cols_to_drop = ["ID de respuesta", "Última página", "Lenguaje inicial", "Semilla", "Fecha de la última acción"]

surveys_path = "./encuestas/pre/"
pre = loader.load_surveys(surveys_path, files_extension, cols_to_drop)
# graphics.display_df(pre, "Pretest")

surveys_path = "./encuestas/post/"
post = loader.load_surveys(surveys_path, files_extension, cols_to_drop)
# graphics.display_df(post, "Postest")

# display(pre)
# display(post)

In [ ]:
import main